### Today we are going to perform the simple classification of the amazon reviews' sentiment.

### Please, download the dataset amazon_baby.csv.

In [207]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.linear_model import LogisticRegression

def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

baby_df = pd.read_csv('amazon_baby.csv')
baby_df.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


## Exercise 1 (data preparation)
a) Remove punctuation from reviews using the given function.   
b) Replace all missing (nan) revies with empty "" string.  
c) Drop all the entries with rating = 3, as they have neutral sentiment.   
d) Set all positive ($\geq$4) ratings to 1 and negative($\leq$2) to -1.

In [208]:
#a)

#short test: 
#baby_df["review"][4] == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'
#no_punctuation = remove_punctuation(baby_df["review"][4])
#no_punctuation == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'


baby_df["review"] = baby_df["review"].fillna(' ')
baby_df["review"] = baby_df["review"].apply(remove_punctuation)
    
baby_df["review"][4] == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'

True

In [209]:
#b)
#done in a)
#short test:
baby_df["review"][38] == baby_df["review"][38]

True

In [210]:
#c)
baby_df = baby_df[baby_df["rating"] != 3]
#short test:
sum(baby_df["rating"] == 3)

0

In [211]:
#d) 
for i in range(0, 6):
    if i < 3:
        baby_df["rating"].replace(i, -1,inplace=True)
    baby_df["rating"].replace(i, 1, inplace=True)


#short test:
sum(baby_df["rating"]**2 != 1)

0

## CountVectorizer
In order to analyze strings, we need to assign them numerical values. We will use one of the simplest string representation, which transforms strings into the $n$ dimensional vectors. The number of dimensions will be the size of our dictionary, and then the values of the vector will represent the number of appereances of the given word in the sentence.

In [212]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
reviews_train_example = ["We like apples",
                   "We hate oranges",
                   "I adore bananas",
                   "We like like apples and oranges",
                   "They dislike bananas"]

X_train_example = vectorizer.fit_transform(reviews_train_example)

print(vectorizer.get_feature_names())
print(X_train_example.todense())



['adore', 'and', 'apples', 'bananas', 'dislike', 'hate', 'like', 'oranges', 'they', 'we']
[[0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 0 1]
 [1 0 0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 2 1 0 1]
 [0 0 0 1 1 0 0 0 1 0]]


In [213]:
reviews_test_example = ["They like bananas",
                   "We hate oranges bananas and apples",
                   "We love bananas"] #New word!

X_test_example = vectorizer.transform(reviews_test_example)

print(X_test_example.todense())

[[0 0 0 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 0 0 0 0 1]]


We should acknowledge few facts. Firstly, CountVectorizer does not take order into account. Secondly, it ignores one-letter words (this can be changed during initialization). Finally, for test values, CountVectorizer ignores words which are not in it's dictionary.

## Exercise 2 
a) Split dataset into training and test sets.     
b) Transform reviews into vectors using CountVectorizer. 

In [214]:
#a)
from sklearn.model_selection import train_test_split

X = baby_df[['review']]
y = baby_df[['rating']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=44)


In [215]:
#b)
vectorizer = CountVectorizer()
X_train_vector = vectorizer.fit_transform(X_train.review)
X_test_vector = vectorizer.transform(X_test.review)


## Exercise 3 
a) Train LogisticRegression model on training data (reviews processed with CountVectorizer, ratings as they were).   
b) Print 10 most positive and 10 most negative words.

In [216]:
#a)
model = LogisticRegression()
model.fit(X_train_vector, y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [247]:
#b)
import collections
wordlist = dict(zip(model.coef_[0], vectorizer.get_feature_names()))
ordered = collections.OrderedDict(sorted(wordlist.items()))

print("Most negative words (smallest coefficients):")
print(list(ordered.items())[:10])

print("Most positive words (largest coefficients):")
print(list(ordered.items())[-10:])
#hint: model.coef_, vectorizer.get_feature_names()

<class 'numpy.ndarray'>
Most negative words (smallest coefficients):
[(-2.8388371513773256, 'dissapointed'), (-2.667759953310932, 'disappointing'), (-2.623779299893747, 'worthless'), (-2.61552675446856, 'worst'), (-2.561291597626928, 'poorly'), (-2.527658385225024, 'useless'), (-2.4869896446347073, 'pointless'), (-2.37370765999488, 'shame'), (-2.345338276921371, 'nope'), (-2.3049635785711846, 'poor')]
Most positive words (largest coefficients):
[(1.8041099828039122, 'amazed'), (1.8243934757350688, 'downside'), (1.837174217113765, 'con'), (1.856128176650343, 'perfect'), (1.8664020996871644, 'ply'), (1.903830414981174, 'lifesaver'), (1.9812292554939557, 'pleasantly'), (2.0805982879172413, 'awesome'), (2.0852926603341166, 'rich'), (2.1472796462448587, 'thankful')]


## Exercise 4 
a) Predict the sentiment of test data reviews.   
b) Predict the sentiment of test data reviews in terms of probability.   
c) Find five most positive and most negative reviews.   
d) Calculate the accuracy of predictions.

In [234]:
#a)
y_pred_sentiment = model.predict(X_test_vector)

In [235]:
#b)
sentiments = model.predict_proba(X_test_vector)
sentiments
#hint: model.predict_proba()

array([[1.41705648e-05, 9.99985829e-01],
       [9.99999987e-01, 1.31914228e-08],
       [2.39613914e-05, 9.99976039e-01],
       ...,
       [3.22430601e-02, 9.67756940e-01],
       [1.58786708e-03, 9.98412133e-01],
       [6.06906354e-04, 9.99393094e-01]])

In [236]:
#c) 
tmp = sorted(zip(sentiments, X_test),
    key = lambda x: x[0][1]
)

for i in tmp[-5:]:
    print(i[0])
    print(i[1][:80])

print('...')

for i in tmp[:5][::-1]:
    print(i[0])
    print(i[1][:80])
#hint: use the results of b)

[1.41705648e-05 9.99985829e-01]
review
...
[1.41705648e-05 9.99985829e-01]
review


In [241]:
#d) 
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred_sentiment)
print(accuracy)

0.932362935906522


## Exercise 5
In this exercise we will limit the dictionary of CountVectorizer to the set of significant words, defined below.


a) Redo exercises 2-5 using limited dictionary.   
b) Check the impact of all the words from the dictionary.   
c) Compare accuracy of predictions and the time of evaluation.

In [249]:
#a)
significant_words = ['love','great','easy','old','little','perfect','loves','well','able','car','broke','less','even','waste',
                     'disappointed','work','product','money','would','return']


X = baby_df[['review']]
y = baby_df[['rating']]
X_train_, X_test_, y_train_, y_test_ = train_test_split(X, y, test_size=0.33, random_state=44)

vectorizer_ = CountVectorizer()
vectorizer_.fit(significant_words)
X_train_vector_ = vectorizer_.fit_transform(X_train_.review)
X_test_vector_ = vectorizer_.transform(X_test_.review)


In [ ]:
model_ = LogisticRegression()
model_.fit(X_train_vector_, y_train_)

significant_wordlist = dict(zip(model_.coef_[0], vectorizer_.get_feature_names()))
ordered_ = collections.OrderedDict(sorted(significant_wordlist.items()))

print("Most negative words (smallest coefficients):")
print(list(ordered_.items())[:10])

print("Most positive words (largest coefficients):")
print(list(ordered_.items())[-10:])

In [244]:
y_pred_sentiment_2 = model.predict(X_test_vector)
sentiments2 = model.predict_proba(X_test_vector)

tmp = sorted(zip(sentiments, X_test),
    key = lambda x: x[0][1]
)

for i in tmp[-5:]:
    print(i[0])
    print(i[1][:80])

print('...')

for i in tmp[:5][::-1]:
    print(i[0])
    print(i[1][:80])


[1.41705648e-05 9.99985829e-01]
review
...
[1.41705648e-05 9.99985829e-01]
review


In [223]:
#b)


In [242]:
#c)
accuracy2 = accuracy_score(y_test, y_pred_sentiment)

print('Accuracy for original dictionary: {}'.format(accuracy))
print('Accuracy for limited dictionary: {}'.format(accuracy2))
#hint: %time, %timeit

Accuracy for original dictionary: 0.932362935906522
Accuracy for limited dictionary: 0.932362935906522
